In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

# Parameters
n_trials = 100  # number of trials
p_correct = 0.75  # proportion of correct trials (true probability)
n_simulations = 5

# Simulate the number of correct trials
for _ in range(n_simulations):
    simulated_correct_trials = np.random.binomial(n=n_trials, p=p_correct)
    simulated_accuracy = simulated_correct_trials/n_trials
    print(f"num_correct={simulated_correct_trials}, pct_correct={simulated_accuracy:3.2f}")

In [ ]:
def fisherz(r, eps=1e-5):
    return np.arctanh(r-eps)

def fisherz_inv(z):
    return np.tanh(z)

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy import stats

def run_simulation(p_correct, n_trials, n_simulations=10000, tol=.05):
    # Simulate the number of correct trials for each n_trials value over 10,000 simulations
    simulated_correct_trials = np.random.binomial(n=n_trials, 
                                                  p=p_correct, 
                                                  size=(n_trials.shape[0], n_simulations))
    prop_correct = simulated_correct_trials/n_trials

    # Compute the 95% confidence interval (2.5th and 97.5th percentiles) for each row
    ci_lower = np.percentile(prop_correct, 2.5, axis=1)  # 2.5th percentile
    ci_upper = np.percentile(prop_correct, 97.5, axis=1)  # 97.5th percentile

    # Combine results into a single array for easy display
    confidence_intervals = np.vstack((ci_lower, ci_upper)).T

    # Print the confidence intervals for each n_trials
    lower_cutoff = p_correct - tol
    upper_cutoff = p_correct + tol
    results = defaultdict(list)
    for i, n in enumerate(n_trials.flatten()):
        scores = prop_correct[i]
        above_cuttoff = (scores > upper_cutoff).sum()
        below_cuttoff = (scores < lower_cutoff).sum()
        p_exceeds_tol = (above_cuttoff + below_cuttoff) / len(scores)
        
        results['p_true'].append(p_correct)
        results['n_trials'].append(n)
        results['mean'].append(scores.mean())
        results['min_score'].append(scores.min())
        results['max_score'].append(scores.max())
        results['sem'].append(stats.sem(scores))
        results['lower_ci'].append(confidence_intervals[i, 0])
        results['upper_ci'].append(confidence_intervals[i, 1])
        results['p_exceeds_tol'].append(p_exceeds_tol)
    
    results = pd.DataFrame(results)
    
    return results

# Parameters
tol = .05 # our tolerance level
n_trials = np.array([10, 25, 50, 100, 200, 250, 275, 300, 400, 1000])[:, np.newaxis]  # multiple n_trials values
n_simulations = 10000  # number of times to run the simulation for each n_trials value
results1 = run_simulation(.50, n_trials, n_simulations, tol)
results1

In [ ]:
results2 = run_simulation(.75, n_trials, n_simulations, tol)
results2

In [ ]:
results3 = run_simulation(.90, n_trials, n_simulations, tol)
results3

In [ ]:
all_results = [results1, results2, results3]

In [ ]:
# plt.figure(figsize=(4, 3))
fig, axes = plt.subplots(1, len(all_results), figsize=(12, 4))

for idx,results in enumerate(all_results):
    p_correct = results.iloc[0].p_true
    ax = axes[idx]
    ax = sns.lineplot(data = results, x="n_trials", y="lower_ci", label="lower_ci", ax=ax)
    ax = sns.lineplot(data = results, x="n_trials", y="upper_ci", label="upper_ci", ax=ax)
    ax.axhline(y=p_correct, color='gray', linestyle='--');
    ax.set_ylim([.0, 1.1])
    if (idx==0):
        ax.set_ylabel('proportion correct', fontsize=14, labelpad=10);
    else:
        ax.set_ylabel('', fontsize=14, labelpad=10);
    ax.set_xlabel('number of trials', fontsize=14, labelpad=10);
    ax.set_title(f"95% CI over {n_simulations} simulations\nEstimating true accuracy={p_correct}", pad=10);
    ax.legend(loc='lower right')

# ax = axes[1]
# results = results2
# ax = sns.lineplot(data = results, x="n_trials", y="lower_ci", label="lower_ci", ax=ax)
# ax = sns.lineplot(data = results, x="n_trials", y="upper_ci", label="upper_ci", ax=ax)
# ax.axhline(y=p_correct, color='gray', linestyle='--', label='Reference line at y=0.5');
# ax.set_ylim([.0, 1.1])
# ax.set_ylabel('proportion correct', fontsize=14, labelpad=10);
# ax.set_xlabel('number of trials', fontsize=14, labelpad=10);
# ax.set_title(f"95% CI over {n_simulations} simulations\nEstimating {p_correct} accuracy from n trials", pad=10);
# ax.legend(loc='lower right')

plt.savefig("simulation_results.png", dpi=300, bbox_inches='tight')

In [ ]:
results

In [ ]:
for n, ci, scores in zip(n_trials, confidence_intervals, prop_correct):
    g = sns.displot(scores)
    ax = plt.gca()
    ax.set_xlim([0,1.2])
    ax.set_ylim([0,3000])
    ax.set_title(f"Number of trials={n[0]}, 95% CI: [{ci[0]:3.2f},{ci[1]:3.2f}]")